In [ ]:
import pandas as pd
import datetime, pytz
import random

### Read users

In [ ]:
users = pd.read_excel("../../data/users_processed.xlsx")

In [ ]:
## Shuffle users 
users = users.sample(frac=1).reset_index(drop=True)

### Read workflow def

In [ ]:
teams_df = pd.read_excel("../../data/workflow.xlsx", sheet_name="teams")
teams = teams_df['Team'].tolist()

steps_df = pd.read_excel("../../data/workflow.xlsx", sheet_name="steps")
step_names = steps_df['Step'].tolist()

users_df = pd.read_excel("../../data/users_processed.xlsx")


In [ ]:
step_names

In [ ]:
def get_users_by_team(user_df, team_name):
    temp_df = users_df.loc[users_df['team'] == team_name]
    temp_df.reset_index(drop=True, inplace=True)
    return temp_df

In [ ]:
#team = 'Sales'

#tmp_df = get_users_by_team(users_df,team)

In [ ]:
team_col_idx = 1
print("Reading teams")
for s in step_names:
    row_df = steps_df.loc[steps_df['Step'] == s]
    team_name = row_df.iat[0,1]
    print(team_name)
    tmp_df = get_users_by_team(users_df,team_name)
    user_count = len(tmp_df.index)
    print("team : "+team_name + " , #users : "+str(user_count))
